In [38]:
!pip install keras==2.13.1 tensorflow==2.13.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Unins

In [1]:
import keras
import torch

2024-10-25 05:56:26.679726: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-25 05:56:26.789816: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-25 05:56:26.790869: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-25 05:56:27.705005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
1

In [3]:
custom_objects = {'ChannelDropout': ChannelDropout}
with keras.saving.custom_object_scope(custom_objects):
    keras_model = keras.models.load_model('Pretraining/Sleep_Models/sleep_model_Fold9.hdf5')

2024-10-25 05:56:34.271358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-25 05:56:34.288725: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
keras_model

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelDropout_pt(nn.Module):
    def __init__(self, rate, noise_shape=None, seed=None):
        super().__init__()
        self.rate = min(1., max(0., rate))
        self.noise_shape = noise_shape
        self.seed = seed
        
    def forward(self, inputs):
        if self.training:
            noise_shape = self.noise_shape or [1, 1, inputs.shape[2]]
            mask = torch.rand(noise_shape, device=inputs.device) > self.rate
            mask = mask.expand_as(inputs).float()
            
            return inputs * mask / (1 - self.rate)
        else:
            return inputs
    
class ModelMDD(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        self.n_timesteps = 3000
        self.n_features = 19

        self.channel_dropout = ChannelDropout_pt(rate=0.25)
        
        # Conv1D layers
        self.conv1 = nn.Conv1d(self.n_features, 5, kernel_size=10, stride=1, padding=0)
        self.conv2 = nn.Conv1d(5, 10, kernel_size=10, stride=1, padding=0)
        self.conv3 = nn.Conv1d(10, 10, kernel_size=10, stride=1, padding=0)
        self.conv4 = nn.Conv1d(10, 15, kernel_size=5, stride=1, padding=0)
        
        # BatchNorm layers
        self.bn1 = nn.BatchNorm1d(5)
        self.bn2 = nn.BatchNorm1d(10)
        self.bn3 = nn.BatchNorm1d(10)
        self.bn4 = nn.BatchNorm1d(15)
        
        # Dense layers
        self.fc1 = nn.Linear(15 * 184, 64)  # 184 is calculated based on the input size and conv/pool operations
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        
        self.dropout = nn.AlphaDropout(dropout)
        
    def forward(self, x):
        # Swap axes as PyTorch uses (N, C, L) format
        x = x.permute(0, 2, 1)
        
        # channel dropout
        x = self.channel_dropout(x)

        # Conv layers
        x = F.elu(self.conv1(x))
        x = F.max_pool1d(x, 2)
        x = self.bn1(x)
        
        x = F.elu(self.conv2(x))
        x = F.max_pool1d(x, 2)
        x = self.bn2(x)
        
        x = F.elu(self.conv3(x))
        x = F.max_pool1d(x, 2)
        x = self.bn3(x)
        
        x = F.elu(self.conv4(x))
        x = F.max_pool1d(x, 2)
        x = self.bn4(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Dense layers
        x = self.dropout(x)
        x = F.elu(self.fc1(x))
        x = self.dropout(x)
        x = F.elu(self.fc2(x))
        x = self.dropout(x)
        x = F.softmax(self.fc3(x), dim=1)
        
        return x

pytorch_model = ModelMDD()


In [32]:
keras_model.layers[-1]

In [33]:
# Function to transfer weights from Keras to PyTorch
def transfer_weights(keras_model, pytorch_model):
    conv_layer = 1
    batchnorm_layer = 1
    lin_layer = 1
    for keras_layer in keras_model.layers:
        print(keras_layer)
        if isinstance(keras_layer, keras.src.layers.convolutional.conv1d.Conv1D):
            # Convert Conv1D weights
            print('conv{}'.format(conv_layer))
            keras_weights, keras_bias = keras_layer.get_weights()
            pytorch_layer = getattr(pytorch_model, 'conv{}'.format(conv_layer))
            pytorch_layer.weight.data = torch.FloatTensor(keras_weights.transpose(2, 1, 0))
            pytorch_layer.bias.data = torch.FloatTensor(keras_bias)
            conv_layer += 1
        
        elif isinstance(keras_layer, keras.src.layers.normalization.batch_normalization.BatchNormalization):
            # Convert BatchNorm weights
            print('bn{}'.format(batchnorm_layer))
            keras_gamma, keras_beta, keras_running_mean, keras_running_var = keras_layer.get_weights()
            pytorch_layer = getattr(pytorch_model, 'bn{}'.format(batchnorm_layer))
            pytorch_layer.weight.data = torch.FloatTensor(keras_gamma)
            pytorch_layer.bias.data = torch.FloatTensor(keras_beta)
            pytorch_layer.running_mean.data = torch.FloatTensor(keras_running_mean)
            pytorch_layer.running_var.data = torch.FloatTensor(keras_running_var)
            batchnorm_layer += 1
        
        elif isinstance(keras_layer, keras.src.layers.core.dense.Dense):
            # Convert Dense layer weights
            print('fc{}'.format(lin_layer))
            keras_weights, keras_bias = keras_layer.get_weights()
            pytorch_layer = getattr(pytorch_model, 'fc{}'.format(lin_layer))
            pytorch_layer.weight.data = torch.FloatTensor(keras_weights.T)
            pytorch_layer.bias.data = torch.FloatTensor(keras_bias)
            lin_layer += 1



In [34]:
transfer_weights(keras_model, pytorch_model)

conv1
bn1
conv2
bn2
conv3
bn3
conv4
bn4
fc1
fc2
fc3


In [35]:
pytorch_model

ModelMDD(
  (channel_dropout): ChannelDropout_pt()
  (conv1): Conv1d(19, 5, kernel_size=(10,), stride=(1,))
  (conv2): Conv1d(5, 10, kernel_size=(10,), stride=(1,))
  (conv3): Conv1d(10, 10, kernel_size=(10,), stride=(1,))
  (conv4): Conv1d(10, 15, kernel_size=(5,), stride=(1,))
  (bn1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=2760, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (dropout): AlphaDropout(p=0.5, inplace=False)
)

In [36]:
# Optional: Verify a few weights
keras_conv1_weight = keras_model.layers[1].get_weights()[0]
pytorch_conv1_weight = pytorch_model.conv1.weight.data.numpy()

print("Keras Conv1 weight shape:", keras_conv1_weight.shape)
print("PyTorch Conv1 weight shape:", pytorch_conv1_weight.shape)
print("First few values match:")
print("Keras:", keras_conv1_weight[0, 0, :5])
print("PyTorch:", pytorch_conv1_weight[0, 0, :5])


Keras Conv1 weight shape: (10, 19, 5)
PyTorch Conv1 weight shape: (5, 19, 10)
First few values match:
Keras: [-0.02743783  0.03102693  0.21714279 -0.03915716 -0.1352543 ]
PyTorch: [-0.02743783  0.17163071  0.09439639  0.01808764 -0.04544811]


In [37]:
import numpy as np

np.random.seed(2)
arr = np.random.random(size=(1, 3000, 19))
# arr = np.zeros((1, 3000, 19))

torch_arr = torch.FloatTensor(arr)

torch.manual_seed(0)
pytorch_model.eval()
pytorch_model(torch_arr)

tensor([[7.5101e-15, 3.3722e-10, 3.5841e-04, 9.9964e-01, 2.7456e-16]],
       grad_fn=<SoftmaxBackward0>)

In [38]:
keras_arr = tf.convert_to_tensor(arr)
# keras.utils.set_random_seed(0)
keras_model(keras_arr)

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[0.0000000e+00, 7.4883410e-08, 1.2655842e-09, 0.0000000e+00,
        9.9999988e-01]], dtype=float32)>